If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
! nvidia-smi

Sun Apr  2 10:12:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install datasets transformers[sentencepiece] sacrebleu #torch==1.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
# ! wget https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
# ! protoc --python_out=. sentencepiece_model.proto
# import sentencepiece_model_pb2

--2022-11-23 05:24:44--  https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13451 (13K) [text/plain]
Saving to: ‘sentencepiece_model.proto’

sentencepiece_model 100%[===================>]  13.14K  --.-KB/s    in 0s      

2022-11-23 05:24:45 (66.3 MB/s) - ‘sentencepiece_model.proto’ saved [13451/13451]



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


# Fine-tuning a model on a translation task

In [ ]:
model_checkpoint = 'Elaine/m2m100_pruned_ta'

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric
metric = load_metric("sacrebleu")

<ipython-input-6-29406150f423>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
# RTAM > RZHO > RNAN/RYUE
train_lang = 'ryue'
data_files = {"train": "/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MonoBitext/ryue_monobitext_250k.json",
              "validation": "/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MonoBitext/valid_ryue.json",
              "test": "/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MonoBitext/test_ryue.json"}
raw_datasets = load_dataset("json", data_files=data_files)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e5e7335df3d3d63b/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 250000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 290
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 290
    })
})

In [ ]:
raw_datasets["test"][0]

{'translation': {'en': "Document `%s' has the wrong type of root node (, should be )",
  'mcsw': "Dokumen ' % s ' have jenis mod root nod yang salah ( , sepatutnya )",
  'lang_switch': ['en']}}

Show a randomly sampled dataframe

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["test"])

,translation
0,"{'en': 'Imagine doing that for someone who has no other shot.', 'mcsw': 'Imagine doing that for someone who has no other shot .', 'lang_switch': ['en']}"
1,"{'en': 'There's an answer for why you knew to warn everyone... the bridge was going to collapse, and minutes later, it did.', 'mcsw': 'Pasti ada jawapan kenapa kau berikan amaran pada semua jambatan tu akan runtuh , dan the last itu ia benar-benar terjadi .', 'lang_switch': ['en']}"
2,"{'en': 'This amazing cockatoo will reveal your future.', 'mcsw': 'Burung yang menakjubkan ini will be mengungkapkan masa depan anda .', 'lang_switch': ['en']}"
3,"{'en': 'I know that doesn't make me a good man, but I tried to do the right thing.', 'mcsw': 'Akutahuitutidakakan membuatkanku becoming seorang yang baik , . . . . . tapiakucubamelakukan perkara yang betul .', 'lang_switch': ['en']}"
4,"{'en': 'I lost a lot of good men, so I know what you and every vet feels now.', 'mcsw': 'Saya kehilangan ramai kawan baik so I tahu apa yang awak dan setiap orang rasa sekarang .', 'lang_switch': ['en']}"


## Preprocessing the data

### Modifying tokenizer for extra language 'MCSW'

In [ ]:
from transformers import AutoTokenizer

# https://github.com/TartuNLP/m2m-100-finetune/blob/main/tok.py
def _update_tok_lang_dicts(tokenizer):
  tokenizer.id_to_lang_token = dict(list(tokenizer.id_to_lang_token.items()) + list(tokenizer.added_tokens_decoder.items()))
  tokenizer.lang_token_to_id = dict(list(tokenizer.lang_token_to_id.items()) + list(tokenizer.added_tokens_encoder.items()))
  tokenizer.lang_code_to_token = { k.replace("_", ""): k for k in tokenizer.additional_special_tokens }
  tokenizer.lang_code_to_id = { k.replace("_", ""): v for k, v in tokenizer.lang_token_to_id.items() }

def get_extended_m2m100_tokenizer(pretrained_model, new_lang_list):
  lang_toks = ["__" + lang + "__" for lang in new_lang_list]
  tokenizer = AutoTokenizer.from_pretrained(pretrained_model, additional_special_tokens = lang_toks)
  _update_tok_lang_dicts(tokenizer)
  return tokenizer

# tokenizer = get_extended_m2m100_tokenizer(model_checkpoint, ['mcsw'])
# tokenizer.src_lang = "mcsw"
tokenizer = get_extended_m2m100_tokenizer(model_checkpoint, [train_lang])
print(len(tokenizer))
tokenizer.src_lang = train_lang
tokenizer.tgt_lang = "en"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


44977


In [ ]:
tokenizer.vocab_size

44971

Test the tokenizer:

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[3, 35378, 4, 903, 1632, 149357, 38, 2], [3, 3293, 83, 15700, 149357, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

Preprocess samples: truncated to the maximum length accepted by the model

In [ ]:
max_input_length = 128
max_target_length = 128
prefix = ""
source_lang = train_lang
target_lang = 'en'

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
raw_datasets['train'][:3]

{'translation': [{'en': "I want grandpa's soul to reborn inside my body",
   'mcsw': 'Saya hendak jiwa datuk for dilahirkan semula di dalam badan saya',
   'lang_switch': ['en']},
  {'en': 'Yes. Yes, that one is "The Hunchback of Notre Dame."',
   'mcsw': 'Ya , that that adalah " The Hunchback of Notre Dame . "',
   'lang_switch': ['en']},
  {'en': 'I was never sick, not one day.',
   'mcsw': 'Aku tidak pernah sakit , satu hari not even.',
   'lang_switch': ['en']}]}

In [ ]:
preprocess_function(raw_datasets['train'][:2])

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[44929, 7891, 22689, 28480, 561, 357, 190, 2991, 29084, 20918, 39, 904, 4077, 1748, 2], [44929, 3494, 217, 8867, 8867, 1862, 33, 1365, 8414, 204, 10923, 415, 25589, 33065, 233, 33, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[44887, 200, 8400, 7279, 783, 12, 54, 25391, 126, 14911, 2193, 42925, 1579, 18486, 2], [44887, 32765, 5, 32765, 4, 8867, 7623, 115, 33, 5885, 8414, 204, 10923, 415, 25589, 33065, 1550, 2]]}

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/250000 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/290 [00:00<?, ? examples/s]

Map:   0%|          | 0/290 [00:00<?, ? examples/s]

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model.resize_token_embeddings(len(tokenizer))
print("No. of parameters: {}M".format(sum(p.numel() for p in model.parameters())/1_000_000))

No. of parameters: 398.775296M


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### Start Training

#### RTAM

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
# model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
datatype = 'monobitext_rtam'
args = Seq2SeqTrainingArguments(
    f"{model_name}-freeze100-finetuned-{source_lang}-to-{target_lang}-{datatype}",
    # evaluation_strategy = "epoch",
    # save_strategy = "epoch",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 2500,
    save_steps = 2500,
    logging_steps = 2500,
    warmup_steps = 1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics #None
)

# Freeze the first 10 layers
for i, param in enumerate(model.parameters()):
    if i < 100:
        param.requires_grad = False


trainer.train()
trainer.push_to_hub()

Cloning https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-rtam-to-en-monobitext_rtam into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
2500,3.329600,2.106315,28.721500,23.837900
5000,2.645200,2.152783,29.940000,24.855200
7500,2.462400,2.169572,29.612600,24.675900


Upload file runs/Apr16_11-54-59_990afa17f55d/events.out.tfevents.1681646110.990afa17f55d.588.0:   0%|         …

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-rtam-to-en-monobitext_rtam
   8de1da7..8d95aad  main -> main

   8de1da7..8d95aad  main -> main

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-rtam-to-en-monobitext_rtam
   8d95aad..af9f7ff  main -> main

   8d95aad..af9f7ff  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-rtam-to-en-monobitext_rtam/commit/8d95aada2c808c5522fa289f66f40f254648df77'

In [ ]:
import datetime
# Save the loss values to the CSV file using the pandas library
temp_df = pd.DataFrame(trainer.state.log_history)
temp_df['training_type'] = datatype
temp_df['time'] = datetime.datetime.now()
temp_df.to_csv(f'/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MonoBitext/{datatype}_training_loss.csv', index=False)

eval_results = trainer.evaluate()
eval_score = eval_results['eval_bleu']
print(eval_score)
# Save the score to the CSV file using the pandas library
# csv_path = 'eval_scores.csv'
# create_csv_file(csv_path)
with open(f'/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MonoBitext/{datatype}_eval_score.csv', 'a') as f:
    f.write(f"{batch_size},{eval_score}\n")

28.7215


#### RZHO

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
# model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch_size = 8
# model_name = model_checkpoint.split("/")[-1]
datatype = 'monobitext_rzho'
args = Seq2SeqTrainingArguments(
    "m2m100_pruned_ta-freeze100-finetuned-rzho-to-en-monobitext",
    # evaluation_strategy = "epoch",
    # save_strategy = "epoch",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 2500,
    save_steps = 2500,
    logging_steps = 2500,
    warmup_steps = 1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics #None
)

# Freeze the first 10 layers
for i, param in enumerate(model.parameters()):
    if i < 100:
        param.requires_grad = False


trainer.train()
trainer.push_to_hub()

Cloning https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-rzho-to-en-monobitext into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
2500,2.490900,2.345398,29.352800,26.493100
5000,1.694900,2.341046,29.899700,27.372400
7500,1.537100,2.362036,29.242400,26.993100


Upload file runs/Apr17_14-07-14_a4b8f70df9cf/events.out.tfevents.1681740445.a4b8f70df9cf.153.0:   0%|         …

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-rzho-to-en-monobitext
   71593a0..fd8a3aa  main -> main

   71593a0..fd8a3aa  main -> main

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-rzho-to-en-monobitext
   fd8a3aa..63b683a  main -> main

   fd8a3aa..63b683a  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-rzho-to-en-monobitext/commit/fd8a3aaad2c62863d0174b9081c48af6d10f8247'

In [ ]:
import datetime
import pandas as pd
# Save the loss values to the CSV file using the pandas library
temp_df = pd.DataFrame(trainer.state.log_history)
temp_df['training_type'] = datatype
temp_df['time'] = datetime.datetime.now()
temp_df.to_csv(f'/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MonoBitext/{datatype}_training_loss.csv', index=False)

eval_results = trainer.evaluate()
eval_score = eval_results['eval_bleu']
print(eval_score)
# Save the score to the CSV file using the pandas library
# csv_path = 'eval_scores.csv'
# create_csv_file(csv_path)
with open(f'/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MonoBitext/{datatype}_eval_score.csv', 'a') as f:
    f.write(f"{batch_size},{eval_score}\n")

#### RNAN

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
# model.resize_token_embeddings(len(tokenizer))
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch_size = 8
# model_name = model_checkpoint.split("/")[-1]
datatype = 'monobitext_rnan'
args = Seq2SeqTrainingArguments(
    "m2m100_pruned_ta-freeze100-finetuned-rnan-to-en-monobitext",
    # evaluation_strategy = "epoch",
    # save_strategy = "epoch",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 2500,
    save_steps = 2500,
    logging_steps = 2500,
    warmup_steps = 1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics #None
)

# Freeze the first 100 layers
for i, param in enumerate(model.parameters()):
    if i < 100:
        param.requires_grad = False


trainer.train()
trainer.push_to_hub()

Cloning https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-rnan-to-en-monobitext into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
2500,2.536700,2.623008,22.773000,28.696600
5000,2.022100,2.622117,23.656100,26.896600


Step,Training Loss,Validation Loss,Bleu,Gen Len
2500,2.536700,2.623008,22.773000,28.696600
5000,2.022100,2.622117,23.656100,26.896600
7500,1.884200,2.613051,23.595200,28.034500


To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-rnan-to-en-monobitext
   1ee3f58..b498f79  main -> main

   1ee3f58..b498f79  main -> main

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-rnan-to-en-monobitext
   b498f79..3a1ba34  main -> main

   b498f79..3a1ba34  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-rnan-to-en-monobitext/commit/b498f79308e7cf76dd6510b49d169db6de74f4c8'

In [ ]:
import datetime
import pandas as pd
# Save the loss values to the CSV file using the pandas library
temp_df = pd.DataFrame(trainer.state.log_history)
temp_df['training_type'] = datatype
temp_df['time'] = datetime.datetime.now()
temp_df.to_csv(f'/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MonoBitext/{datatype}_training_loss.csv', index=False)

eval_results = trainer.evaluate()
eval_score = eval_results['eval_bleu']
print(eval_score)
# Save the score to the CSV file using the pandas library
# csv_path = 'eval_scores.csv'
# create_csv_file(csv_path)
with open(f'/content/drive/MyDrive/ColabNotebooks/MCSW_Research/Data/MonoBitext/{datatype}_eval_score.csv', 'a') as f:
    f.write(f"{batch_size},{eval_score}\n")

23.5952


#### RYUE

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
# model.resize_token_embeddings(len(tokenizer))
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch_size = 8
# model_name = model_checkpoint.split("/")[-1]
datatype = 'monobitext_ryue'
args = Seq2SeqTrainingArguments(
    "m2m100_pruned_ta-freeze100-finetuned-ryue-to-en-monobitext",
    # evaluation_strategy = "epoch",
    # save_strategy = "epoch",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 2500,
    save_steps = 2500,
    logging_steps = 2500,
    warmup_steps = 1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics #None
)

# Freeze the first 100 layers
for i, param in enumerate(model.parameters()):
    if i < 100:
        param.requires_grad = False


trainer.train()
trainer.push_to_hub()

Cloning https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-ryue-to-en-monobitext into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu,Gen Len
2500,2.416600,2.642178,23.269300,26.558600
5000,1.898800,2.639679,23.878400,27.713800
7500,1.759700,2.667382,24.660900,27.244800


Upload file runs/Apr18_11-37-15_9ece42d43021/events.out.tfevents.1681817845.9ece42d43021.527.0:   0%|         …

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-ryue-to-en-monobitext
   4c71c2f..74dc8e9  main -> main

   4c71c2f..74dc8e9  main -> main

To https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-ryue-to-en-monobitext
   74dc8e9..fe037ae  main -> main

   74dc8e9..fe037ae  main -> main



'https://huggingface.co/Elaine/m2m100_pruned_ta-freeze100-finetuned-ryue-to-en-monobitext/commit/74dc8e9c67563200119afce48ca6a93445bae00b'

# Testing

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
new_model = "Elaine/m2m100_pruned_ta-finetuned-mcsw-to-en"
my_model = AutoModelForSeq2SeqLM.from_pretrained(new_model)
my_tokenizer = get_extended_m2m100_tokenizer(new_model, ['mcsw'])
my_tokenizer.src_lang = "mcsw" # 'ms'

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Elaine--m2m100_pruned_ta-finetuned-mcsw-to-en/snapshots/27cb31a21d2b213fd36e48f64b4092849f3d57f1/config.json
Model config M2M100Config {
  "_name_or_path": "Elaine/m2m100_pruned_ta-finetuned-mcsw-to-en",
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "M2M100ForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.05,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.05,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 200,
  "max_position_embeddings": 1024,
  "model_type": "m2m_100",
  "num_beams": 5,
  "nu

In [ ]:
def my_translation(text, model, tokenizer):
  encoded_text = tokenizer(text, return_tensors="pt")
  generated_tokens = model.generate(**encoded_text,
                                    forced_bos_token_id=tokenizer.get_lang_id("en"),
                                    max_new_tokens=40)
  print("Ori: {}\nTrans: {}".format(text, tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)))

Model Before Finetune

In [ ]:
my_translation('Letnan, kita memiliki dua the enemy direction. 300 meter di sebelah utara ',
               model, tokenizer)
my_translation('dan sekarang saya yakin... kalian ada sesuatu yang in fact. milik kami.',
               model, tokenizer)
my_translation('Aku boleh pinjam tentera from rakan aku , Yin Po .',
               model, tokenizer)
my_translation('Planet aku bernama Lorien but the earth is tempat ku sekarang .',
               model, tokenizer)

Ori: Letnan, kita memiliki dua the enemy direction. 300 meter di sebelah utara 
Trans: ['Letnan, kita have dua the enemy direction. 300 meters di sebelah utara']
Ori: dan sekarang saya yakin... kalian ada sesuatu yang in fact. milik kami.
Trans: ['And now I am sure... you have something that actually belongs to us.']
Ori: Aku boleh pinjam tentera from rakan aku , Yin Po .
Trans: ['I can pinjam tentera from rakan aku, Yin Po.']
Ori: Planet aku bernama Lorien but the earth is tempat ku sekarang .
Trans: ['Planet I am named Lorien but the earth is tempat ku now.']


In [ ]:
my_translation('Comfortable if we tahu dengan siapa kita bercakap .',
               model, tokenizer)
my_translation('Yeah . Orang dewasa coming ke taman... dan tak pulang hidup-hidup ',
               model, tokenizer)

Ori: Comfortable if we tahu dengan siapa kita bercakap .
Trans: ['Comfortable if we know with whom we talk.']
Ori: Yeah . Orang dewasa coming ke taman... dan tak pulang hidup-hidup 
Trans: ['People come to the park and don’t come home live-life']


Model After Finetune

In [ ]:
my_translation('Letnan, kita memiliki dua the enemy direction. 300 meter di sebelah utara ',
               my_model, my_tokenizer)
my_translation('dan sekarang saya yakin... kalian ada sesuatu yang in fact. milik kami.',
               my_model, my_tokenizer)
my_translation('Aku boleh pinjam tentera from rakan aku , Yin Po .',
               my_model, my_tokenizer)
my_translation('Planet aku bernama Lorien but the earth is tempat ku sekarang .',
               my_model, my_tokenizer)

Ori: Letnan, kita memiliki dua the enemy direction. 300 meter di sebelah utara 
Trans: ['Letnan, we have two enemy directions. 300 meters north']
Ori: dan sekarang saya yakin... kalian ada sesuatu yang in fact. milik kami.
Trans: ["And now I'm sure... you have something that really belongs to us."]
Ori: Aku boleh pinjam tentera from rakan aku , Yin Po .
Trans: ['I can borrow a military from my friend, Yin Po.']
Ori: Planet aku bernama Lorien but the earth is tempat ku sekarang .
Trans: ['My planet is Lorien, but the earth is my place now.']


In [ ]:
my_translation('Comfortable if we tahu dengan siapa kita bercakap .',
               my_model, my_tokenizer)
my_translation('Yeah. Orang dewasa coming ke taman... dan tak pulang hidup-hidup ',
               my_model, my_tokenizer)

Ori: Comfortable if we tahu dengan siapa kita bercakap .
Trans: ["Comfortable if we know with whom we're talking."]
Ori: Yeah. Orang dewasa coming ke taman... dan tak pulang hidup-hidup 
Trans: ["Yes, adults come to the park... and don't get home alive."]
